_____
***

<img width='700' src="https://user-images.githubusercontent.com/8030363/108961534-b9a66980-7634-11eb-96e2-cc46589dcb8c.png" style="vertical-align:middle">

***
***

**Author:** [TJCallahan](https://mail.google.com/mail/u/0/?view=cm&fs=1&tf=1&to=callahantiff@gmail.com)  
**GitHub Repository:** [PheKnowLator](https://github.com/callahantiff/PheKnowLator/wiki)  
**Current Release:** **[`v2.0.0`](https://github.com/callahantiff/PheKnowLator/wiki/v2.0.0)**

<br>

**Objective:** Knowledge graphs provide meaningful ways to integrate heterogeneous biological data and represent complex biological mechanisms. This work seeks to explore the utility of incorporating existing knowledge of molecular mechanism from ontologies, publicly available data, and the literature to construct a biomedical knowledge graph that models unbiased molecular mechanisms of human disease.

<a target="_blank" href="https://user-images.githubusercontent.com/8030363/103158881-11813b00-4780-11eb-8b45-5063765e7645.png"> <img src="https://user-images.githubusercontent.com/8030363/103158881-11813b00-4780-11eb-8b45-5063765e7645.png"></a> 

(*Click Figure to Enlarge Image in Current Browser Tab*)

<br>

***
***

## Notebook Purpose
**Wiki Page:** **[`Release v2.0.0`](https://github.com/callahantiff/PheKnowLator/wiki/v2.0.0)**

<br>

**Purpose:** This notebook serves as a `main` file for the PheKnowLator project. This scripts walks through this program step-by-step and generates the knowledge graph shown above. There is also a command line version of this file ([`main.py`](https://github.com/callahantiff/PheKnowLator/blob/master/main.py)). Please see the [README](https://github.com/callahantiff/PheKnowLator/blob/master/README.md) for more information.

<br>

**Assumptions:**
1. Hyperlinks to all downloaded and generated data sources are provided through [this](https://console.cloud.google.com/storage/browser/pheknowlator/release_v2.0.0?project=pheknowlator) dedicated Google Cloud Storage Bucket. <u>This includes examples, from prior builds, of the required input documents mentioned below</u>.     
2. Make sure that the following input documents have been constructed (see the [Dependencies Wiki](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies) for more information):  
  - [`resource_info.txt`](https://github.com/callahantiff/PheKnowLator/blob/master/resources/resource_info.txt)
  - [`ontology_source_list.txt`](https://github.com/callahantiff/PheKnowLator/blob/master/resources/ontology_source_list.txt)
  - [`edge_source_list.txt`](https://github.com/callahantiff/PheKnowLator/blob/master/resources/edge_source_list.txt)   

3. Prepare [relations](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#relations-data) and [node metadata](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata) files prior to running the scripts.  

4. Select a knowledge graph build type (i.e. `full`, `partial`, or `post-closure`) and construction method (i.e. `instance-based` or `subclass-based`).  

<br>

***
### Table of Contents
***
The three primary steps involved in building a knowledge graph are `Downloading Data Sources`, `Creating Edge Lists`, and `Building the knowledge graphs`.

* [Data Sources](#data-source)  
* [Create Edge Lists](#create-edges)  
* [Build Knowledge Graph](#build-kg)  

***

***

_____
### Set-Up Environment

In [20]:
# import needed libraries
import glob
import json
import pandas
import ray
import time

# import module
from pkt_kg import downloads, edge_list, knowledge_graph

***
## Download Data Sources <a class="anchor" id="data-source"></a>

**Wiki Page:** **[`Dependencies`](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies)**  

**Purpose:**
This portion of this portion of the algorithm is to download:
1. [Download Ontology Data](#download-ontology-data)  
2. [Download Edge Data](#download-edge-data)   

<br>

**Input Files:**
  - [`resource_info.txt`](https://github.com/callahantiff/PheKnowLator/blob/master/resources/resource_info.txt)
  - [`ontology_source_list.txt`](https://github.com/callahantiff/PheKnowLator/blob/master/resources/ontology_source_list.txt)
  - [`edge_source_list.txt`](https://github.com/callahantiff/PheKnowLator/blob/master/resources/edge_source_list.txt)

<br>

**Assumptions:**  
- All sources used to construct our knowledge graph need to be preprocessed and ready to download prior to running this code. All mapping, filtering, and label data have been generated prior to this step. For assistance with creating these datasets, see the [`Data_Preparation.ipynb`](https://github.com/callahantiff/PheKnowLator/blob/master/notebooks/Data_Preparation.ipynb) Jupyter Notebook.  
- All downloaded and generated data sources for all PheKnowLator builds can be accessed through [this](https://console.cloud.google.com/storage/browser/pheknowlator/release_v2.0.0?project=pheknowlator) dedicated Google Cloud Storage Bucket.


***
***
### Ontology Data  <a class="anchor" id="download-ontology-data"></a>
Ontologies are the core data structure used when building PheKnowLator. In the figure above, ontology data are shown in yellow boxes.

In [1]:
from pkt_kg.downloads import OntData, LinkedData
from pkt_kg.edge_list import CreatesEdgeList
from pkt_kg.knowledge_graph import FullBuild, PartialBuild, PostClosureBuild

In [2]:
ont = OntData('resources/ontology_source_list.txt', 'resources/resource_info.txt')
#not a function of OntData
#ont._writes_source_metadata_locally()

In [3]:
ont.data_path

'resources/ontology_source_list.txt'

In [4]:
ont.parses_resource_file()

In [5]:
ont.gets_data_type()

'Ontology Data'

In [6]:
ont.source_list

{'disease': 'resources/ontologies/mondo_with_imports.owl',
 'napdichem': 'resources/ontologies/chebi_lite_merged_with_imports.owl',
 'protein': 'resources/ontologies/pr_with_imports.owl',
 'pathway': 'resources/ontologies/pw_with_imports.owl',
 'relation': 'resources/ontologies/ro_with_imports_AD_mods.owl',
 'go': 'resources/ontologies/go_with_imports.owl',
 'chemical': 'resources/ontologies/chebi_lite_with_imports.owl',
 'phenotype': 'resources/ontologies/hp_with_imports.owl',
 'anatomy': 'resources/ontologies/ext_with_imports.owl',
 'cell': 'resources/ontologies/clo_with_imports.owl',
 'genomic': 'resources/ontologies/so_with_imports.owl',
 'oae': 'resources/ontologies/oae_merged_with_imports.owl'}

In [7]:
ont.data_files = ont.source_list
ont.generates_source_metadata()


*** Generating Metadata ***



100%|██████████| 12/12 [00:00<00:00, 60061.63it/s]


In [35]:
dir(ont)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__metaclass__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_writes_source_metadata_locally',
 'data_files',
 'data_path',
 'data_type',
 'downloads_data_from_url',
 'extracts_edge_metadata',
 'generates_source_metadata',
 'gets_data_type',
 'metadata',
 'parses_resource_file',
 'resource_data',
 'resource_dict',
 'resource_info',
 'source_list']

In [63]:
ont._writes_source_metadata_locally()

100%|██████████| 12/12 [00:00<00:00, 54887.29it/s]


In [8]:
ont.resource_info

["chemical-disease|:;MESH_;|class-class|RO_0002606|http://purl.obolibrary.org/obo/|http://purl.obolibrary.org/obo/|t|1;4|0:./resources/processed_data/MESH_CHEBI_MAP.txt;1:./resources/processed_data/DISEASE_MONDO_MAP.txt|5;!=;''|None",
 "chemical-gene|;MESH_;|class-entity|RO_0002434|http://purl.obolibrary.org/obo/|http://www.ncbi.nlm.nih.gov/gene/|t|1;4|0:./resources/processed_data/MESH_CHEBI_MAP.txt|9;affects;not in x|6;==;Homo sapiens::5;.startswith('gene');",
 'chemical-gobp|:;MESH_;GO_|class-class|RO_0002436|http://purl.obolibrary.org/obo/|http://purl.obolibrary.org/obo/|t|1;5|0:./resources/processed_data/MESH_CHEBI_MAP.txt|8;<=;1.04e-47|3;==;Biological Process',
 'chemical-gocc|:;MESH_;GO_|class-class|RO_0002436|http://purl.obolibrary.org/obo/|http://purl.obolibrary.org/obo/|t|1;5|0:./resources/processed_data/MESH_CHEBI_MAP.txt|8;<=;1.04e-47|3;==;Cellular Component',
 'chemical-gomf|:;MESH_;GO_|class-class|RO_0002436|http://purl.obolibrary.org/obo/|http://purl.obolibrary.org/obo/|t

In [65]:
ont.resource_data

'resources/resource_info.txt'

<br>

### Edge Data   <a class="anchor" id="download-edge-data"></a>
In PheKnowLator, classes are nodes that originate from ontologies. Class data sources are Linked Data sources that are used to create edges in the knowledge graph and thus can connect to other class data sources. Sometimes we want to add data that is not already part of an ontology. In that case, data either be added as an `instance` of an existing ontology class or as its own `owl:class` by being added to the knowledge graph as a `subclass` of an existing `owl:class`.

In [14]:
edges = LinkedData('resources/edge_source_list_TC.txt', 'resources/resource_info.txt')
#do this the first time to download all the edge data
edges.downloads_data_from_url()
edges.writes_source_metadata_locally()


*** Downloading Data: edge_source_list_TC to "resources/edge_data/" ***



  0%|          | 0/33 [00:00<?, ?it/s]


Edge: chemical-disease

Edge: chemical-gene

Edge: chemical-gobp


  9%|▉         | 3/33 [00:04<00:43,  1.45s/it]


Edge: chemical-gocc

Edge: chemical-gomf


...


Edge: variant-phenotype


100%|██████████| 33/33 [01:07<00:00,  2.06s/it]



*** Generating Metadata ***



100%|██████████| 33/33 [00:00<00:00, 35112.13it/s]


AttributeError: 'LinkedData' object has no attribute 'writes_source_metadata_locally'

In [9]:
edges = LinkedData('resources/edge_source_list.txt', 'resources/resource_info.txt')

In [10]:
edges.resource_data

'resources/resource_info.txt'

In [11]:
edges.parses_resource_file()

In [12]:
edges.source_list

{'chemical-disease': 'resources/edge_data/chemical-disease_CTD_chemicals_diseases.tsv',
 'chemical-gene': 'resources/edge_data/chemical-gene_CTD_chem_gene_ixns.tsv',
 'chemical-gobp': 'resources/edge_data/chemical-gobp_CTD_chem_go_enriched.tsv',
 'chemical-gocc': 'resources/edge_data/chemical-gocc_CTD_chem_go_enriched.tsv',
 'chemical-gomf': 'resources/edge_data/chemical-gomf_CTD_chem_go_enriched.tsv',
 'chemical-pathway': 'resources/edge_data/chemical-pathway_ChEBI2Reactome_All_Levels.txt',
 'chemical-protein': 'resources/edge_data/chemical-protein_CTD_chem_gene_ixns.tsv',
 'chemical-phenotype': 'resources/edge_data/CTD_chemicals_diseases.tsv',
 'disease-phenotype': 'resources/edge_data/phenotype.hpoa',
 'gene-disease': 'resources/edge_data/gene-disease_curated_gene_disease_associations.tsv',
 'gene-gene': 'resources/edge_data/gene-gene_COMBINED.DEFAULT_NETWORKS.BP_COMBINING.txt',
 'gene-pathway': 'resources/edge_data/gene-pathway_CTD_genes_pathways.tsv',
 'gene-phenotype': 'resources

In [13]:
edges.data_files = edges.source_list
edges.generates_source_metadata()


*** Generating Metadata ***



100%|██████████| 37/37 [00:00<00:00, 51626.50it/s]


In [20]:
dir(edges)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__metaclass__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_writes_source_metadata_locally',
 'data_files',
 'data_path',
 'data_type',
 'downloads_data_from_url',
 'extracts_edge_metadata',
 'generates_source_metadata',
 'gets_data_type',
 'metadata',
 'parses_resource_file',
 'resource_data',
 'resource_dict',
 'resource_info',
 'source_list']

In [14]:
edges._writes_source_metadata_locally()

100%|██████████| 37/37 [00:00<00:00, 46715.61it/s]


In [15]:
edges.data_path

'resources/edge_source_list.txt'

In [16]:
edges.source_list.keys()

dict_keys(['chemical-disease', 'chemical-gene', 'chemical-gobp', 'chemical-gocc', 'chemical-gomf', 'chemical-pathway', 'chemical-protein', 'chemical-phenotype', 'disease-phenotype', 'gene-disease', 'gene-gene', 'gene-pathway', 'gene-phenotype', 'gene-protein', 'gene-rna', 'gobp-pathway', 'pathway-gocc', 'pathway-gomf', 'protein-anatomy', 'protein-catalyst', 'protein-cofactor', 'protein-cell', 'protein-gobp', 'protein-gocc', 'protein-gomf', 'protein-pathway', 'protein-protein', 'rna-anatomy', 'rna-cell', 'rna-protein', 'variant-disease', 'variant-gene', 'variant-phenotype', 'chemical-transporter', 'chemical-molecule', 'chemical-substrate', 'chemical-inhibitor'])

In [17]:
ont.data_files

{'disease': 'resources/ontologies/mondo_with_imports.owl',
 'napdichem': 'resources/ontologies/chebi_lite_merged_with_imports.owl',
 'protein': 'resources/ontologies/pr_with_imports.owl',
 'pathway': 'resources/ontologies/pw_with_imports.owl',
 'relation': 'resources/ontologies/ro_with_imports_AD_mods.owl',
 'go': 'resources/ontologies/go_with_imports.owl',
 'chemical': 'resources/ontologies/chebi_lite_with_imports.owl',
 'phenotype': 'resources/ontologies/hp_with_imports.owl',
 'anatomy': 'resources/ontologies/ext_with_imports.owl',
 'cell': 'resources/ontologies/clo_with_imports.owl',
 'genomic': 'resources/ontologies/so_with_imports.owl',
 'oae': 'resources/ontologies/oae_merged_with_imports.owl'}

In [18]:
edges.data_files

{'chemical-disease': 'resources/edge_data/chemical-disease_CTD_chemicals_diseases.tsv',
 'chemical-gene': 'resources/edge_data/chemical-gene_CTD_chem_gene_ixns.tsv',
 'chemical-gobp': 'resources/edge_data/chemical-gobp_CTD_chem_go_enriched.tsv',
 'chemical-gocc': 'resources/edge_data/chemical-gocc_CTD_chem_go_enriched.tsv',
 'chemical-gomf': 'resources/edge_data/chemical-gomf_CTD_chem_go_enriched.tsv',
 'chemical-pathway': 'resources/edge_data/chemical-pathway_ChEBI2Reactome_All_Levels.txt',
 'chemical-protein': 'resources/edge_data/chemical-protein_CTD_chem_gene_ixns.tsv',
 'chemical-phenotype': 'resources/edge_data/CTD_chemicals_diseases.tsv',
 'disease-phenotype': 'resources/edge_data/phenotype.hpoa',
 'gene-disease': 'resources/edge_data/gene-disease_curated_gene_disease_associations.tsv',
 'gene-gene': 'resources/edge_data/gene-gene_COMBINED.DEFAULT_NETWORKS.BP_COMBINING.txt',
 'gene-pathway': 'resources/edge_data/gene-pathway_CTD_genes_pathways.tsv',
 'gene-phenotype': 'resources

<br>

***

## Create Edge Lists <a class="anchor" id="create-edges"></a>

**Wiki Page:** **[`Data Sources`](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources)**

<br>

**Purpose:** The code below will take the dictionaries of processed data described above and use them to create edge lists for each of the edge types specified in the [`resource_info.txt`](https://github.com/callahantiff/PheKnowLator/blob/master/resources/resource_info.txt). Each edge list will be appended to a nested dictionary (see details below).

<br>

**Assumptions:**  
1. All `ontology` and `edge` data sources have been downloaded.   

2. All code in the [`Data_Preparation.ipynb`](https://github.com/callahantiff/PheKnowLator/blob/master/notebooks/Data_Preparation.ipynb) Jupyter Notebook has been run. This Notebook contains code needed to generate all mapping, filtering, and label data.

<br>

**Output:** `Master_Edge_List_Dict.json`. Below is an example of what the `Master Edge Dictionary` contains for each processed resource:  
```python
master_edges = {'chemical-disease'  :
                {'source_labels'    : ';MESH_;',
                 'data_type'        : 'class-class',
                 'edge_relation'    : 'RO_0002606',
                 'uri'              : ('http://purl.obolibrary.org/obo/',
                                       'http://purl.obolibrary.org/obo/'),
                 'delimiter'        : '#',
                 'column_idx'       : '1;4',
                 'identifier_maps'  : '0:./MESH_CHEBI_MAP.txt;1:disease-dbxref-map',
                 'evidence_criteria': "5;!=;' ",
                 'filter_criteria'  : 'None',
                 'edge_list'        : ['...']}
```

***

In [21]:
# set-up environment for parallel processing -- even if running program serially these steps are needed
import psutil
cpus = psutil.cpu_count(logical=True)
ray.init()

/home/sanya/.conda/envs/sanya_kg/lib/python3.6/site-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)


{'node_ip_address': '130.49.206.138',
 'raylet_ip_address': '130.49.206.138',
 'redis_address': '130.49.206.138:6379',
 'object_store_address': '/tmp/ray/session_2022-03-30_12-41-46_342243_3555/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-03-30_12-41-46_342243_3555/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-03-30_12-41-46_342243_3555',
 'metrics_export_port': 56008,
 'node_id': 'c23430e46bbfab3ff7dde3fa63aab2bb9ab0360dc7a9f7e0a5d77941'}

In [22]:
# combine data sources
combined_edges = dict(edges.data_files, **ont.data_files)
resource_info_loc = './resources/resource_info.txt'

# initialize edge dictionary class
master_edges = CreatesEdgeList(data_files=combined_edges, source_file=resource_info_loc)
master_edges.runs_creates_knowledge_graph_edges(source_file=resource_info_loc, data_files=combined_edges, cpus=cpus)

(pid=22131) Finished Edge: gene-protein (gene = 19318, protein = 19135); 19523 unique edges
(pid=22131) Finished Edge: rna-protein (rna = 44199, protein = 19197); 44202 unique edges
(pid=22135) Finished Edge: gobp-pathway (gobp = 478, pathway = 671); 671 unique edges
(pid=22145) Finished Edge: gene-pathway (gene = 10371, pathway = 1860); 107025 unique edges
(pid=22137) Finished Edge: gene-rna (gene = 25529, rna = 179874); 182717 unique edges
(pid=22133) Finished Edge: gene-disease (gene = 5037, disease = 4431); 12738 unique edges
(pid=22136) Finished Edge: gene-phenotype (gene = 6785, phenotype = 1531); 23525 unique edges
(pid=22143) Finished Edge: chemical-pathway (chemical = 2169, pathway = 2232); 29248 unique edges
(pid=22138) Finished Edge: disease-phenotype (disease = 11864, phenotype = 9964); 427477 unique edges
(pid=22143) Finished Edge: protein-cell (protein = 10045, cell = 128); 75318 unique edges
(pid=22136) Finished Edge: rna-cell (rna = 14044, cell = 130); 65180 unique edge

**Preview Master Edge Data**  
Generate a table that includes each `edge-type`, its primary `relation`, example identifiers, and count of unique edges.

In [23]:
master_edges = json.load(open('resources/Master_Edge_List_Dict.json', 'r'))
master_edges.keys()

dict_keys(['chemical-transporter', 'chemical-gobp', 'pathway-gomf', 'chemical-phenotype', 'chemical-molecule', 'chemical-substrate', 'protein-protein', 'variant-phenotype', 'rna-cell', 'gene-protein', 'chemical-inhibitor', 'pathway-gocc', 'chemical-gocc', 'chemical-disease', 'protein-gocc', 'gene-disease', 'chemical-gomf', 'gene-pathway', 'rna-anatomy', 'protein-catalyst', 'gene-gene', 'disease-phenotype', 'chemical-pathway', 'variant-gene', 'protein-pathway', 'protein-gomf', 'variant-disease', 'chemical-gene', 'protein-anatomy', 'gobp-pathway', 'protein-cofactor', 'gene-rna', 'rna-protein', 'chemical-protein', 'protein-gobp', 'gene-phenotype', 'protein-cell'])

In [24]:
# # read in master edge dictionary
#master_edges = json.load(open('resources/Master_Edge_List_Dict.json', 'r'))

# read in relation data
relation_data = open('./resources/relations_data/RELATIONS_LABELS.txt').readlines()
relation_dict = {x.split('\t')[0]: x.split('\t')[1].strip('\n') for x in relation_data}

# print basic stats on each resource
edge_data = [[key,
              relation_dict[master_edges[key]['edge_relation']],
              ', '.join(master_edges[key]['edge_list'][0]),
              len(master_edges[key]['edge_list'])]
             for key in master_edges.keys()]

# convert dict to pandas df for nice printing
df = pandas.DataFrame(edge_data, columns = ['Edge Type', 'Relation', 'Example Edge', 'Unique Edges']) 
df                

,Edge Type,Relation,Example Edge,Unique Edges
0,chemical-transporter,transports,"CHEBI_51141, PR_Q14242",90
1,chemical-gobp,molecularly interacts with,"CHEBI_35299, GO_0048856",304686
2,pathway-gomf,has function,"R-HSA-8876283, GO_0015370",2424
3,chemical-phenotype,is substance that treats,"CHEBI_701, HP_0001250",108366
4,chemical-molecule,molecularly interacts with,"CHEBI_6030, PR_P08684",391
5,chemical-substrate,is substrate of,"CHEBI_8871, PR_P08684",514
6,protein-protein,molecularly interacts with,"PR_P84085, PR_O15020",618069
7,variant-phenotype,causes or contributes to condition,"rs121909005, HP_0012873",2764
8,rna-cell,located in,"ENST00000442999, UBERON_0000002",65180
9,gene-protein,has gene product,"9988, PR_Q9Y222",19523


<br><br>

***

## Build Knowledge Graph  <a class="anchor" id="build-kg"></a>
**Wiki Pages:**  
- **[`KG-Construction`](https://github.com/callahantiff/PheKnowLator/wiki/KG-Construction)**  
- **[`relations-data`](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#relations-data)**  
- **[`node-metadata`](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata)** 

**Jupyter Notebooks:**  
- [`Data_Preparation.ipynb`](https://github.com/callahantiff/PheKnowLator/blob/master/notebooks/Data_Preparation.ipynb)  
[`Ontology_Cleaning.ipynb`](https://github.com/callahantiff/PheKnowLator/blob/master/notebooks/Ontology_Cleaning.ipynb)  


<br>

**Assumptions:**  
- <u>Construction Approach</u>. If using the `subclass-based` construction approach, please make sure that a `pickled` dictionary mapping each non-ontology data node to an existing ontology class is created and added to the `./resources/knowledge_graph` directory (please see [here](https://github.com/callahantiff/PheKnowLator/tree/master/resources/knowledge_graphs#construction-method) for additional information).   
- <u>Relations Data</u>. If inverse relation data is going to be used to build the knowledge graph, that it has been generated and added to the `./resources/relations_data` directory (please see [here](https://github.com/callahantiff/PheKnowLator/blob/master/resources/relations_data/README.md) for additional information).  
- <u>Node Metadata</u>. If node metadata is going to be used to build the knowledge graph, that it has been generated and added to the `./resources/node_metadata` directory (please see [here](https://github.com/callahantiff/PheKnowLator/blob/master/resources/node_data/README.md) for additional information).  
- <u>Decoding OWL Semantics</u>. If decoding OWL-Semantics, please make sure to provide a list of owl:Property types to keep is created and added to the `./resources/knowledge_graph` directory (please see [here](https://github.com/callahantiff/PheKnowLator/wiki/OWL-NETS-2.0) for additional information). 

<br>

**Input:** 
- `Master_Edge_List_Dict.json`  
- Directory of relations data sources - see [here](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#relations-data) for more information
- Directory of node data sources - see [here](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata) for more information

<br>

**Output:** Please see [`Release v2.0.0 Wiki`](https://github.com/callahantiff/PheKnowLator/wiki/v2.0.0) for access to all generated output files.   
- `Knowledge Graph` (`.owl` and Networkx MultiDiGraph `.pkl`)  
- `Class Instance URI-UUID Map` (if "instance" construction approach)   
- `Triple List - Integer`  
- `Triple List - Identifier`  
- `Node Integer-Identifier Map`  
- `Node Attribute Data`  

<br>

The process to build the knowledge graph is somewhat time consuming and can be broken into the following steps:  

1. Merge Ontologies. See [here](https://github.com/callahantiff/PheKnowLator/blob/master/resources/ontologies/README.md) for additional information on how to preprocess the ontologies prior to merging them.    

2. Create Edges. Add edge lists to merged ontologies.  

3. Add Inverse Relations and Node Data. See the [Dependencies](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies) Wiki page for details on how to construct these resources.  

4. Filter OWL Semantics. Filter the knowledge graph with the goal of removing all edges that contain entities that are needed to support owl semantics, but are not biologically meaningful (please see [here](https://github.com/callahantiff/PheKnowLator/wiki/OWL-NETS-2.0) for additional information).

5. Save Edge Lists and Node Metadata. Several versions of the knowledge graph are saved, including: the full knowledge graph (`owl` or Networkx MultiDiGraph `pickle`), triple lists (i.e. integer index and identifier labeled edge lists with a dictionary that maps between the integer indices and node identifiers), and a file of metadata (i.e. identifiers, labels, synonyms, and descriptions) for all nodes in the knowledge graph.  

<br>

**‼ IMPORTANT:**  
- The file containing the merged ontologies is quite large and can take up to 30 minutes to read in.  This is not a limitation of the code directly, but rather a function of the [`RDFLib Library`](https://github.com/RDFLib). While there are other ways to read in this data, we maintain reliance on this library as it is the most user-friendly for non-RDF users.   
- If you'd like to include [node metadata](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata) when building the knowledge graph, please hold off on building the knowledge graph until you have generated the node data. For details on how to do this see the [node metadata](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata) section of the `Dependencies` Wiki Page or help for help with generating the data, please see the []() section of the [`Data_Preparation.ipynb`](https://github.com/callahantiff/PheKnowLator/blob/master/notebooks/Data_Preparation.ipynb) Jupyter Notebook.

***


In [27]:
# specify input arguments
build = 'full'
construction_approach = 'instance'
add_node_data_to_kg = 'yes'
add_inverse_relations_to_kg = 'yes'
decode_owl_semantics = 'yes'
kg_directory_location = './resources/knowledge_graphs'

In [ ]:
# construct knowledge graphs, run sed command before this
if build == 'partial':
    kg = PartialBuild(construction=construction_approach,
                      node_data=add_node_data_to_kg,
                      inverse_relations=add_inverse_relations_to_kg,
                      decode_owl=decode_owl_semantics,
                      cpus=cpus,
                      write_location=kg_directory_location)
elif build == 'post-closure':
    kg = PostClosureBuild(construction=construction_approach,
                          node_data=add_node_data_to_kg,
                          inverse_relations=add_inverse_relations_to_kg,
                          decode_owl=decode_owl_semantics,
                          cpus=cpus,
                          write_location=kg_directory_location)
else:
    kg = FullBuild(construction=construction_approach,
                   node_data=add_node_data_to_kg,
                   inverse_relations=add_inverse_relations_to_kg,
                   decode_owl=decode_owl_semantics,
                   cpus=cpus,
                   write_location=kg_directory_location)

kg.construct_knowledge_graph()
ray.shutdown()


### Starting Knowledge Graph Build: FULL ###
*** Loading Relations Data ***
Loading and Processing Relation Data
*** Loading Merged Ontologies ***
Merged Ontologies Graph Stats: 9695698 triples, 4086431 nodes, 343 predicates, 534098 classes, 35 individuals, 805 object props, 622 annotation props
*** Loading Node Metadata Data ***
Loading and Processing Node Metadata

Extracting Class and Relation Metadata


100%|██████████| 805/805 [00:00<00:00, 5147.13it/s]


*** Splitting Graph ***
Adding Namespace to BNodes
Creating Logic and Annotation Subsets of Graph


100%|██████████| 906931/906931 [10:24<00:00, 1451.89it/s] 


Annotation Assertions (n=5671912 Triples)
Creating Logic Graph (n=4023786 Triples)


...

  6%|▌         | 1/17 [12:28<3:19:39, 748.69s/it]

Removing owl:disjointWith Axioms
Filtering Triples



 12%|█▏        | 2/17 [15:20<1:42:22, 409.47s/it]

Removing owl:disjointWith Axioms
Filtering Triples



 50%|████▉     | 483855/967901 [05:42<04:43, 1710.12it/s]

In [ ]:
###MODIFIED/ADDITIONS BELOW -- Sanya (09/16/2021)
###Last run - 2022-03-30

In [ ]:
## create dictionary from NodeLabels and serialize it
## do with the pickle file (patched nodes - to remove foreign characters)

In [2]:
import os
import os.path
from tqdm import tqdm
import networkx as nx 
from rdflib import Graph, URIRef, BNode, Namespace, Literal  
from rdflib.namespace import RDF, OWL  

In [1]:
import pandas as pd
import pickle

In [3]:
NX_GRAPH_PATH = '/home/sanya/PheKnowLatorv2/resources/knowledge_graphs/'
OUT_NX_GRAPH_NAME = 'PheKnowLator_v3.0.0_full_instance_inverseRelations_OWLNETS_NodeLabels.nt'

In [4]:
#nodedf = pd.read_csv(NX_GRAPH_PATH+'PheKnowLator_v3.0.0_full_instance_inverseRelations_OWLNETS_NodeLabels.txt', sep='\t')
##nodedf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752949 entries, 0 to 752948
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   entity_type             752838 non-null  object
 1   integer_id              752917 non-null  object
 2   entity_uri              752868 non-null  object
 3   label                   752767 non-null  object
 4   description/definition  752761 non-null  object
 5   synonym                 752711 non-null  object
dtypes: object(6)
memory usage: 34.5+ MB


In [9]:
#nodedf2 = pd.read_csv('/home/sanya/kg_test/pl-build_tc/PheKnowLator_v3.0.0_full_instance_inverseRelations_OWLNETS_NodeLabels.txt', sep='\t')
#nodedf2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 745701 entries, 0 to 745700
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   entity_type             691490 non-null  object
 1   integer_id              745649 non-null  object
 2   entity_uri              745598 non-null  object
 3   label                   691394 non-null  object
 4   description/definition  691394 non-null  object
 5   synonym                 691341 non-null  object
dtypes: object(6)
memory usage: 34.1+ MB


In [4]:
##create nodelabels nt file from pickle dictionary
##save nodelabels as ntriples
##save nodelabels as TSV - use next build
with open('/home/sanya/PheKnowLatorv2/resources/knowledge_graphs/nodeLabels_20220329.pickle', 'rb') as filen:
    nods = pickle.load(filen)
len(nods)

753219

In [5]:
for key in nods:
    print(key)
    print(nods[key])
    break

https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000558233
IPO4-205


In [6]:
##serialize as ntriples
graph  = Graph()
pred_label = URIRef("http://www.w3.org/2000/01/rdf-schema#label")
for key in nods:
    node = URIRef(key)
    nodelabel = nods[key]
    graph.add((node, pred_label, Literal(nodelabel)))

In [7]:
graph.serialize('/home/sanya/PheKnowLatorv2/resources/knowledge_graphs/PheKnowLator_v3.0.0_full_instance_inverseRelations_OWLNETS_NodeLabels.nt', format='nt')

In [10]:
#checking for foreign characters
nodedf2.loc[nodedf2['entity_uri'] == '<http://purl.obolibrary.org/obo/SO_0000704>']

,entity_type,integer_id,entity_uri,label,description/definition,synonym
472103,NODES,4024,<http://purl.obolibrary.org/obo/SO_0000704>,gene,A region (or regions) that includes all of the...,INSDC_feature:gene|INSDC_feature:gene


In [6]:
nodedf.loc[nodedf['entity_uri'] == '<http://purl.obolibrary.org/obo/GO_0002682>']

,entity_type,integer_id,entity_uri,label,description/definition,synonym
686102,NODES,10467,<http://purl.obolibrary.org/obo/GO_0002682>,免疫系统过程调控,"Any process that modulates the frequency, rate...",None


In [14]:
nodedf2.at[1, 'label']

'pantothenic acid metabolic pathway'

In [7]:
import re
import numpy as np
nodes = []

for i in range(len(nodedf.index)):
    label = nodedf.at[i, 'label']
    if isinstance(label, str):
        uri = nodedf.at[i, 'entity_uri']
        if re.search("[\u4e00-\u9FFF]", label):
            nodes.append(uri)

len(nodes)

1974

In [21]:
nodes[10:20]

['<http://purl.obolibrary.org/obo/CLO_0053675>',
 '<http://purl.obolibrary.org/obo/CLO_0051777>',
 '<http://purl.obolibrary.org/obo/CLO_0054281>',
 '<http://purl.obolibrary.org/obo/GO_0051179>',
 '<http://purl.obolibrary.org/obo/CLO_0001952>',
 '<http://purl.obolibrary.org/obo/CLO_0052720>',
 '<http://purl.obolibrary.org/obo/CLO_0051921>',
 '<http://purl.obolibrary.org/obo/CLO_0054124>',
 '<http://purl.obolibrary.org/obo/CLO_0053444>',
 '<http://purl.obolibrary.org/obo/CLO_0051848>']

In [8]:
nodedf.loc[nodedf['entity_uri'] == '<http://purl.obolibrary.org/obo/SO_0000704>']

,entity_type,integer_id,entity_uri,label,description/definition,synonym
226058,NODES,4394,<http://purl.obolibrary.org/obo/SO_0000704>,基因,A region (or regions) that includes all of the...,INSDC_feature:gene|INSDC_feature:gene


In [25]:
nodedf.loc[nodedf2['entity_uri'] == nodes[0]]

,entity_type,integer_id,entity_uri,label,description/definition,synonym
1077,NODES,742600,<http://purl.obolibrary.org/obo/CLO_0052035>,NICR295 细胞,None,None


<br>

***
***

```
@misc{callahan_tj_2019_3401437,
  author       = {Callahan, TJ},
  title        = {PheKnowLator},
  month        = mar,
  year         = 2019,
  doi          = {10.5281/zenodo.3401437},
  url          = {https://doi.org/10.5281/zenodo.3401437}
}
```